## 서브워드텍스트인코더(SubwordTextEncoder)
* SubwordTextEncoder는 텐서플로우를 통해 사용할 수 있는 서브워드 토크나이저
- BPE와 유사한 알고리즘인 Wordpiece Model을 채택하였으며, 패키지를 통해 쉽게 단어들을 서브워드들로 분리가능

```
Tensorflow 2.3+ 버전에서는 tfds.features.text 대신 tfds.deprecated.text라고 작성
```

<br>

### IMDB 리뷰 토큰화

In [4]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

train_df = pd.read_csv('IMDb_Reviews.csv')

<br>

#### `tensorflow_datasets.deprecated.text.SubwordTextEncoder.build_from_corpus(토큰화할 데이터, target_vocab_size)` :  서브워드들로 이루어진 단어 집합(Vocabulary)를 생성하고, 각 서브워드에 고유한 정수를 부여

In [7]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(train_df['review'], target_vocab_size=2**13)

<br>

#### `서브워드토크나이저객체.subwords` : 토큰화된 서브워드 확인

In [8]:
print(tokenizer.subwords[:100])

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 't_', 'as_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', 'are_', ' (', 'have_', 'his_', 'film_', 'not_', 'be_', 'you_', 'ing_', ' "', 'ed_', 'it', 'd_', 'an_', 'at_', 'by_', 'he_', 'one_', 'who_', 'from_', 'y_', 'or_', 'e_', 'like_', 'all_', '" ', 'they_', 'so_', 'just_', 'has_', ') ', 'about_', 'her_', 'out_', 'This_', 'some_', 'movie', 'ly_', 'film', 'very_', 'more_', 'It_', 'what_', 'would_', 'when_', 'if_', 'good_', 'up_', 'which_', 'their_', 'only_', 'even_', 'my_', 'really_', 'had_', 'can_', 'no_', 'were_', 'see_', '? ', 'she_', 'than_', '! ', 'there_', 'been_', 'get_', 'into_', 'will_', ' - ', 'much_', 'n_', 'because_', 'ing']


<br>

#### `서브워드토크나이저객체.encode()` : 입력한 데이터에 대해서 정수 인코딩을 수행한 결과확인

<br>

* 정수 인코딩을 수행한 결과와 비교

In [9]:
print(train_df['review'][20], end='\n\n')

print('Tokenized sample question: {}'.format(tokenizer.encode(train_df['review'][20])))

Pretty bad PRC cheapie which I rarely bother to watch over again, and it's no wonder -- it's slow and creaky and dull as a butter knife. Mad doctor George Zucco is at it again, turning a dimwitted farmhand in overalls (Glenn Strange) into a wolf-man. Unfortunately, the makeup is virtually non-existent, consisting only of a beard and dimestore fangs for the most part. If it were not for Zucco and Strange's presence, along with the cute Anne Nagel, this would be completely unwatchable. Strange, who would go on to play Frankenstein's monster for Unuiversal in two years, does a Lenny impression from "Of Mice and Men", it seems.<br /><br />*1/2 (of Four)

Tokenized sample question: [1590, 4162, 132, 7107, 1892, 2983, 578, 76, 12, 4632, 3422, 7, 160, 175, 372, 2, 5, 39, 8051, 8, 84, 2652, 497, 39, 8051, 8, 1374, 5, 3461, 2012, 48, 5, 2263, 21, 4, 2992, 127, 4729, 711, 3, 1391, 8044, 3557, 1277, 8102, 2154, 5681, 9, 42, 15, 372, 2, 3773, 4, 3502, 2308, 467, 4890, 1503, 11, 3347, 1419, 8127, 2

<br>

#### `서브워드토크나이저객체.decode()` : 인코딩된 정수에 대하여 문자열로 디코딩

<br>

* 임의로 선택한 짧은 문장에 대해서 정수 인코딩 결과를 확인하고, 이를 다시 역으로 디코딩

In [12]:
sample_string = "It's mind-blowing to me that this film was even made."

# 인코딩한 결과
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 : [137, 8051, 8, 910, 8057, 2169, 36, 7, 103, 13, 14, 32, 18, 79, 681, 8058]
기존 문장 : It's mind-blowing to me that this film was even made.


<br>

#### `서브워드토크나이저객체.vocab_size` : 단어 집합의 크기를 확인

In [13]:
print('단어 집합의 크기(Vocab size) :', tokenizer.vocab_size)

단어 집합의 크기(Vocab size) : 8268


In [14]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

137 ----> It
8051 ----> '
8 ----> s 
910 ----> mind
8057 ----> -
2169 ----> blow
36 ----> ing 
7 ----> to 
103 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
79 ----> even 
681 ----> made
8058 ----> .


<br>

* 기존 예제 문장 중 even이라는 단어에 임의로 xyz라는 3개의 글자를 추가
- **현재 토크나이저가 even이라는 단어를 이미 하나의 서브워드로 인식하고 있는 상황에서 나머지 xyz를 어떻게 분리하는지를 확인**

In [15]:
# 앞의 sample_string에 even 뒤에 임의로 xyz 추가
sample_string = "It's mind-blowing to me that this film was evenxyz made."

# 인코딩
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 : [137, 8051, 8, 910, 8057, 2169, 36, 7, 103, 13, 14, 32, 18, 7974, 8132, 8133, 997, 681, 8058]
기존 문장 : It's mind-blowing to me that this film was evenxyz made.


* **evenxyz에서 even을 독립적으로 분리하고,**

  **xyz는 훈련 데이터에서 하나의 단어로서 등장한 적이 없으므로 각각 전부 분리**

In [16]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

137 ----> It
8051 ----> '
8 ----> s 
910 ----> mind
8057 ----> -
2169 ----> blow
36 ----> ing 
7 ----> to 
103 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
7974 ----> even
8132 ----> x
8133 ----> y
997 ----> z 
681 ----> made
8058 ----> .


<br>

### 네이버 영화 리뷰 토큰화

In [17]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds

In [18]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')

In [19]:
print(train_data.isnull().sum())

id          0
document    5
label       0
dtype: int64


In [20]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [22]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(train_data['document'], target_vocab_size=2**13)

<br>

In [23]:
print(train_data['document'][20])

나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님


In [24]:
print('Tokenized sample question: {}'.format(tokenizer.encode(train_data['document'][20])))

Tokenized sample question: [669, 4700, 17, 1749, 8, 96, 131, 1, 48, 2239, 4, 7466, 32, 1274, 2655, 7, 80, 749, 1254]


<br>


In [25]:
sample_string = train_data['document'][21]

# 인코딩
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 : [570, 892, 36, 584, 159, 7091, 201]
기존 문장 : 보면서 웃지 않는 건 불가능하다


In [26]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

570 ----> 보면서 
892 ----> 웃
36 ----> 지 
584 ----> 않는 
159 ----> 건 
7091 ----> 불가능
201 ----> 하다


<br>

<br>

## 허깅페이스 토크나이저(Huggingface Tokenizer)
- 자연어 처리 스타트업 허깅페이스가 개발한 패키지 tokenizers는 자주 등장하는 서브워드들을 하나의 토큰으로 취급하는 다양한 서브워드 토크나이저를 제공

<br>

```python
pip install tokenizers
```

<br>

### BERT의 워드피스 토크나이저(BertWordPieceTokenizer)
- 구글이 공개한 딥 러닝 모델 BERT에는 WordPiece Tokenizer가 사용
- 허깅페이스는 해당 토크나이저를 직접 구현하여 tokenizers라는 패키지를 통해 버트워드피스토크나이저(BertWordPieceTokenizer)를 제공

<br>

#### 데이터 로드

In [28]:
import pandas as pd
import urllib.request
from tokenizers import BertWordPieceTokenizer

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f4eb680ffd0>)

#### 데이터 전처리
- ratings.txt라는 파일을 데이터프레임으로 로드한 후, 결측값을 제거
- naver_review.txt라는 파일로 저장

In [29]:
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')

with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

<br>

#### BERT 워드피스 토크나이저 설정

<br>

#### `tokenizers.BertWordPieceTokenizer(lowercase, strip_accents)` : BERT 워드피스 토크나이저 객체 생성
- `lowercase` : 대소문자를 구분 여부. True일 경우 구분하지 않음
- `strip_accents` : True일 경우 악센트 제거.
  - ex) é → e, ô → o


In [31]:
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)

<br>

#### BERT 워드피스 토크나이저 학습

<br>

#### `'BERT 워드피스 토크나이저 객체'.train(files, vocab_size, limit_alphabet, min_frequency)`
- `files` : 단어 집합을 얻기 위해 학습할 데이터
- `vocab_size` : 단어 집합의 크기
- `limit_alphabet` : 병합 전의 초기 토큰의 허용 개수
- `min_frequency` : 최소 해당 횟수만큼 등장한 쌍(pair)의 경우에만 병합 대상

In [32]:
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

In [33]:
tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency)

<br>

- vocab 저장

In [34]:
# vocab 저장
tokenizer.save_model('./')

['./vocab.txt']

- vocab을 데이터프레임으로 로드

In [36]:
# vocab 로드
df = pd.read_fwf('vocab.txt', header=None)
print(df.shape)
df.head()

(30000, 1)


,0
0,[PAD]
1,[UNK]
2,[CLS]
3,[SEP]
4,[MASK]


<br>

#### `'BERT 워드피스 토크나이저 객체'.encode(문자열)` : 토큰화
* `.ids`는 실질적인 딥 러닝 모델의 입력으로 사용되는 정수 인코딩 결과를 출력
* `.tokens`는 해당 토크나이저가 어떻게 토큰화를 진행했는지를 출력
* `decode(.ids)`는 정수 시퀀스를 문자열로 복원

In [37]:
encoded = tokenizer.encode('아 배고픈데 짜장면먹고싶다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['아', '배고', '##픈', '##데', '짜장면', '##먹고', '##싶다']
정수 인코딩 : [2111, 20630, 3862, 3573, 24680, 7871, 7378]
디코딩 : 아 배고픈데 짜장면먹고싶다


In [38]:
encoded = tokenizer.encode('커피 한잔의 여유를 즐기다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['커피', '한잔', '##의', '여유', '##를', '즐기', '##다']
정수 인코딩 : [12825, 25642, 3242, 12696, 3416, 10784, 3258]
디코딩 : 커피 한잔의 여유를 즐기다


<br>

### 기타 토크나이저
* 이 외 **`ByteLevelBPETokenizer`**, **`CharBPETokenizer`**, **`SentencePieceBPETokenizer`** 등이 존재하며 선택에 따라서 사용

<br>

* **`BertWordPieceTokenizer`** : BERT에서 사용된 워드피스 토크나이저(WordPiece Tokenizer)
* **`CharBPETokenizer`** : 오리지널 BPE
* **`ByteLevelBPETokenizer`** : BPE의 바이트 레벨 버전
* **`SentencePieceBPETokenizer`** : 앞서 본 패키지 센텐스피스(SentencePiece)와 호환되는 BPE 구현체

In [39]:
from tokenizers import ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer

In [40]:
tokenizer = SentencePieceBPETokenizer()
tokenizer.train('naver_review.txt', vocab_size=10000, min_frequency=5)

encoded = tokenizer.encode("이 영화는 정말 재미있습니다.")
print(encoded.tokens)

['▁이', '▁영화는', '▁정말', '▁재미있', '습니다.']
